In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
cookies = {    
    'SID': 'YOUR_SID_HERE',
}

# Установка User-Agent: https://stackoverflow.com/a/10606260/9370853
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/81.0.4044.122 Chrome/81.0.4044.122 Safari/537.36',
}

In [ ]:
# Как задать cookies: https://stackoverflow.com/a/7164897/9370853

def dl_books(page_idx):
    res = requests.get("https://www.litres.ru/pages/my_books_fresh/page-{}/?lite=1".format(page_idx), cookies=cookies, headers=headers)
    html_data = res.content
    return html_data

In [ ]:
my_books_all = []

page_idx = 1
while True:
    soup = BeautifulSoup(dl_books(page_idx), "html.parser")
    
    # Как выбрать элементы по CSS-классу: https://stackoverflow.com/a/5041056/9370853
    books = soup.findAll("div", {"class": "art-item"})
    
    if len(books) == 0:
        break
    for book in books:
#         print(book)
        book_author = book.find("div", {"class": "art__author"})        
        if book_author is None:  # https://www.litres.ru/valentina-kolyvanova/cvetnaya-imperiya-rossiya-do-potryaseniy-fot-19194085/
            book_author = "Default fallback author"
            warnings.warn("A book without an author has appeared")
        else:
            book_author_1 = book_author.find("a")
            if book_author_1 is not None:
                # Получение значения атрибута: https://stackoverflow.com/a/2612577/9370853
                book_author = book_author_1['title']
            else:
                book_author_2 = book_author.select_one("span")
                if book_author_2 is not None:
                    # Получение текста из span-тега: https://stackoverflow.com/questions/38133759/how-to-get-text-from-span-tag-in-beautifulsoup
                    book_author = book_author_2.text

        book_name_and_url = book.find("div", {"class": "art__name"}).find("a")
        if book_name_and_url is not None:
            book_name = book_name_and_url['title']
            book_url = "https://www.litres.ru{0}".format(book_name_and_url['href'])
            my_books_all.append({ 'url': book_url, 'author': book_author, 'name': book_name })
        else:
            warnings.warn("A book without URL has appeared")
    
    print(page_idx)
    page_idx += 1
    
    # Задержка в секундах: https://www.tutorialspoint.com/python/time_sleep.htm
    time.sleep(3)
    
#     break

f = open("my_books_all.html", "w")
for my_book in my_books_all:
    f.write('<a href="{0}" rel="nofollow">{1} - {2}</a><br>\n'.format(my_book['url'], my_book['author'], my_book['name']))
f.close()